### Atividade 

#### Parte Ivan

- Criar arquivo terraform para criação de 3 buckets (raw, trusted e refined) e lambda para enviar arquivos de Raw para Trusted 
- Criar código para subir dados para bucket 
- criar código para enviar de lambda raw para trusted 
- Criar a rotina para reprocessamento
- Entregar arquivo PDF com a descrição dos tratamento e particionamentos realizados 
- Entregar arquivo ZIP com os dados Tratados
- Entregar arquivo ZIP com o script Python utilizado




#### Parte Gustavo e Luis

1) Separar quais arquivos cada um vai tratar

2) Analisar os Arquivos e considerar as transformações necessárias para limpeza,
padronização, enriquecimento de dados e reprocessamento dos dados 
- Sugestão de padronização no sumário abaixo (podém mudar o que preferirem, só alinhem entre si pois os arquivos tem que ter o mesmo tratamento)
- Converter dados de latitude e longitude que estão em graus para decimais (se for o arquivo que tem essa coluna, o professor disponibilizou o codigo)
- Somente considerar voos domésticos
 
3) Os arquivos de Voos e de Reclamações precisam passar por tratamento para que
sejam armazenados no Bucket Trusted de forma que facilitem reprocessamentos
4) Particionar por mes, ou seja, criar um arquivo csv para cada mes, nesse formato que o professor pediu : 
- dados_voo_YYYYMM
- reclamação_YYYYMM


### Avisos:

- Os arquivos csv são muito grandes para mandar p github então podem colocar em uma pasta chamada "dados", eu adicionei um .env que não vai mandar para o git, eu colocaria os dados em uma subpasta chamada raw e mandaria para outra subpasta chamada trusted, mas podem deixar que eu arrumo no final


- Para acessar e rodar os arquivos ipynb usar a extensão do VSCode jupyter notebook da Microsoft
- Esta Prática servirá de base para a nossa próxima aula
- Para o tratamento dos dados podem usar como base o nosso arquivo tratamento.ipynb no github https://github.com/BeiraMar-G2/beira-mar-data-analytics tem muita coisa que pode ser reaproveitada
- A Atividade pode ser feita em Grupo, não esqueça de informar Nome e RA
- Entrega
- Sugestão para nome de arquivo     
- Não esqueça de documentar também o reprocessamento dos arquivos dos meses
de Novembro e Dezembro de 2024

### Sumário de padrões para tratamento de dados:

- Data: DD/MM/YYYY

- Data hora: DD/MM/YYYY HH:MM:SS

- Binário: 0 ou 1

- Strings e colunas: Maiusculas e sem acento: 

- Espaços: Manter apenas um espaço no intervalo entre palavras

- Sexo: M e F

- Delimitador: Vírgula

- Armazenar dados em arquivos csv

In [ ]:
import math
import pandas as pd 
import win32com.client as win32
import unicodedata
import os
import re

### Funções para padronização:

In [57]:
# Formato em 2016-04-29T18:38:08Z
def padronizar_data_hora(df, coluna):

  df[coluna] = pd.to_datetime(df[coluna])
  
  df[coluna] = df[coluna].dt.strftime('%d/%m/%Y %H:%M:%S')
  
  return df


In [58]:
#Formato em MM/DD/AA
def padronizar_data(df, coluna):

  df[coluna] = pd.to_datetime(df[coluna], format='%m/%d/%Y')
  
  df[coluna] = df[coluna].dt.strftime('%d/%m/%Y')
  
  return df

In [59]:
def padronizar_data02(df, coluna):
    # Converte a coluna para o tipo datetime, tratando erros
    df[coluna] = pd.to_datetime(df[coluna], format='%Y-%m-%d', errors='coerce')
    
    # Formata as datas válidas, deixando os NaNs como estão
    df[coluna] = df[coluna].dt.strftime('%d/%m/%Y')
    
    return df


In [60]:
def padronizar_colunas(df):

    df.columns = df.columns.str.upper()
    
    return df

In [61]:
def converter_para_binario(df, coluna):
    mapeamento = {'Yes': 1, 'No': 0}
    df[coluna].replace(mapeamento, inplace=True)
    return df

In [62]:
def funcao_apoio_remover_acentos(texto):
    """Remove apenas os acentos de uma string."""
    if not isinstance(texto, str):
        return texto 

    texto_normalizado = unicodedata.normalize('NFKD', texto)
    texto_sem_acentos = "".join(
        [c for c in texto_normalizado if not unicodedata.combining(c)]
    )
    return texto_sem_acentos

In [63]:
def remover_acentos_no_df(df):
    """
    Remove acentos do cabeçalho e de colunas de texto no DataFrame.
    """
    df.columns = [funcao_apoio_remover_acentos(col) for col in df.columns]
    colunas_de_texto = df.select_dtypes(include=['object'])
    
    for coluna in colunas_de_texto.columns:
        df[coluna] = df[coluna].apply(funcao_apoio_remover_acentos)

    return df

In [64]:
def remover_acentos(df):
    for coluna in df.columns:
        if df[coluna].dtype == 'object':
            df[coluna] = df[coluna].astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    return df


In [65]:
def padronizar_maiusculo(df):
    for coluna in df.columns:
        if df[coluna].dtype == 'object':
            df[coluna] = df[coluna].astype(str).str.upper()
    return df

In [105]:

def padronizar_espacos_colunas(df: pd.DataFrame) -> pd.DataFrame:

    novas_colunas = []
    for coluna in df.columns:
        coluna_sem_acento = ''.join(c for c in unicodedata.normalize('NFD', coluna) 
                                    if unicodedata.category(c) != 'Mn')

        coluna_normalizada = coluna_sem_acento.lower().replace(' ', '_').replace('-', '_')
        novas_colunas.append(coluna_normalizada)
    
    df.columns = novas_colunas
    return df


In [66]:
def dms_para_dd(graus, minutos, segundos, direcao):
    """
    Converte coordenadas de Graus/Minutos/Segundos (DMS) para Graus Decimais (DD).
    Exemplo: dms_para_dd(8, 20, 55, 'S') -> -8.348611
    """
    decimal = graus + (minutos / 60) + (segundos / 3600)
    if direcao in ['S', 'W']:
        decimal = -decimal
    return decimal


def haversine(lat1, lon1, lat2, lon2):
    """
    Calcula a distância entre dois pontos na Terra (em km)
    a partir de suas coordenadas em graus decimais.
    """
    R = 6371.0  # raio médio da Terra em km
    
    # Converter para radianos
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Diferenças
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    

    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    return R * c


### Tratando bases:

In [67]:

base_dir = os.getcwd()

caminho_xls = os.path.join(base_dir, "dados", "raw", "Dados Atividade", "aerodromospublicos.xls")
caminho_xlsx = os.path.join(base_dir, "dados", "raw", "Dados Atividade", "aerodromospublicos.xlsx")

print(caminho_xls)

c:\Users\ivanm\OneDrive - SPTech School\SPTECH\Materias\2ANO\2 Semestre\Analise de Dados\novo analise-de-voos\analise-de-voos\dados\raw\Dados Atividade\aerodromospublicos.xls


In [68]:
def converter_xls_para_xlsx(caminho_relativo_xls):
    base_dir = os.getcwd()
    arquivo_xls = os.path.join(base_dir, caminho_relativo_xls)
    arquivo_xlsx = arquivo_xls.replace(".xls", ".xlsx")
    
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(arquivo_xls)
    wb.SaveAs(arquivo_xlsx, FileFormat=51) 
    wb.Close()
    excel.Application.Quit()

    print(f"Conversão concluída: {arquivo_xlsx}")

# Exemplo de uso
converter_xls_para_xlsx(r"dados\raw\Dados Atividade\aerodromospublicos.xls")

Conversão concluída: c:\Users\ivanm\OneDrive - SPTech School\SPTECH\Materias\2ANO\2 Semestre\Analise de Dados\novo analise-de-voos\analise-de-voos\dados\raw\Dados Atividade\aerodromospublicos.xlsx


In [69]:
df_aero = pd.read_excel('dados/raw/Dados Atividade/aerodromospublicos.xlsx',  header=2)

In [70]:
df_aero

,CÓDIGO OACI,CIAD,NOME,MUNICÍPIO ATENDIDO,UF,LATITUDE,LONGITUDE,ALTITUDE,OPERAÇÃO,OACI,...,FREQ. SEMANAL,r1,r2,REFERÊNCIA(S),RESTRIÇÃO(ÕES),SITUAÇÃO,r1.1,REFERÊNCIA,SITUADO NA AMAZÔNIA LEGAL,OBSERVAÇÕES
0,SBAA,PA0008,CONCEIÇÃO DO ARAGUAIA,CONCEIÇÃO DO ARAGUAIA,PA,8° 20' 55'' S,49° 18' 11'' W,199 m,VFR Diurno/Noturno e IFR Diurno,-,...,-,-,-,-,-,Proibição de operações de pouso de aeronaves d...,PA2019-1362,PA2019-1362,X,-
1,SBAE,SP0010,BAURU/AREALVA,BAURU,SP,22° 9' 28'' S,49° 4' 6'' W,598 m,VFR Diurno/Noturno e IFR Diurno/Noturno,-,...,-,-,-,-,-,-,-,-,NaN,-
2,SBAQ,SP0012,BARTOLOMEU DE GUSMÃO,ARARAQUARA,SP,21° 48' 43'' S,48° 7' 59'' W,711 m,VFR Diurno/Noturno e IFR Diurno/Noturno,-,...,-,-,-,-,-,-,-,-,NaN,-
3,SBAR,SE0001,SANTA MARIA,ARACAJU,SE,10° 59' 7'' S,37° 4' 24'' W,7 m,VFR Diurno/Noturno e IFR Diurno/Noturno,RS,...,NaN,PA2022-7303 / PA2020-0406 / PA2021-4220,-,PA2022-7303 / PA2020-0406 / PA2021-4220,-,-,-,-,NaN,-
4,SBAT,MT0003,PILOTO OSVALDO MARQUES DIAS,ALTA FLORESTA,MT,9° 51' 59'' S,56° 6' 18'' W,289 m,VFR Diurno/Noturno e IFR Diurno/Noturno,-,...,-,-,-,-,-,-,-,-,X,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,SJO9,BA0253,SENHOR DO BONFIM,SENHOR DO BONFIM,BA,10° 31' 7'' S,40° 4' 55'' W,466 m,VFR Diurno,-,...,-,-,-,-,-,-,-,-,NaN,-
500,SBPW,RJ0321,PORTO DO AÇU,SÃO JOÃO DA BARRA,RJ,21° 48' 15'' S,41° 6' 32'' W,5 m,VFR Diurno/Noturno e IFR Diurno/Noturno,-,...,-,-,-,-,-,-,-,-,NaN,-
501,SSBG,RS0091,MUNICIPAL DE BENTO GONÇALVES,BENTO GONÇALVES,RS,29° 8' 51'' S,51° 32' 27'' W,671 m,VFR Diurno,-,...,-,-,-,-,-,Proibição de operações de pouso de aeronaves t...,PA2024-14.530,PA2024-14.530,NaN,-
502,SND4,SC0020,ISMAEL NUNES,SÃO JOAQUIM,SC,28° 14' 44'' S,49° 53' 51'' W,1426 m,VFR Diurno,-,...,-,-,-,-,-,-,-,-,NaN,-


In [ ]:
df_aero = padronizar_colunas(df_aero)
df_aero = remover_acentos_no_df(df_aero)
df_aero = padronizar_maiusculo(df_aero)
df_aero = padronizar_espacos_colunas(df_aero)

In [72]:
df_empresas = pd.read_excel('dados/raw/Dados Atividade/Empresas Aereas.xlsx', engine='openpyxl')

In [ ]:
df_empresas = padronizar_colunas(df_empresas)
df_empresas = remover_acentos_no_df(df_empresas)
df_empresas = padronizar_maiusculo(df_empresas)
df_empresas = padronizar_espacos_colunas(df_empresas)
padronizar_data_hora(df_empresas,'DATA DECISAO OPERACIONAL')
padronizar_data_hora(df_empresas,'VALIDADE OPERACIONAL')



,RAZAO SOCIAL,ICAO,IATA,REPRESENTANTE LEGAL,PAIS SEDE,CNPJ,ENDERECO,CIDADE,UF,CEP,TELEFONE,E-MAIL,DECISAO OPERACIONAL,ATIVIDADES AEREAS,DATA DECISAO OPERACIONAL,VALIDADE OPERACIONAL,ID EMPRESA AEREA
0,"21 AIR, LLC",CSB,2P,VANDERLEI MORELLI,ESTADOS UNIDOS DA AMERICA,00.000.000/0000-00,"AV. FRANCISCO GLICERIO , 2331 – TERREO - SALA...",CAMPINAS,SP,13.023-101,(19) 99596-9494,PERMIT.BRAZIL@GMAIL.COM MORELLIVANDERLEI@GMAIL...,OFICIO NO 190/2024/GEAM EMPRESAS/GEAM/SAS-ANAC...,ESTRANGEIRA NAO REGULAR,16/12/2024 00:00:00,01/11/2025 00:00:00,1007093
1,2 PILOTOS AVIACAO AGRICOLA LTDA,NAN,NAN,NAN,BRASIL,49.683.649/0001-43,"RUA 44, SN, QUADRA 67 LOTE 33 SALA 01 JARDIM ...",ARAGUAINA,TO,77.820-214,.,2PILOTOSAVIACAO@GMAIL.COM,"PORTARIA NO 14.774, DE 7 DE JUNHO DE 2024",AEROAGRICOLA,11/06/2024 00:00:00,NaN,1007649
2,A2 AVIACAO AGRICOLA LTDA.,NAN,..,NAN,BRASIL,22.499.581/0001-90,"RODOVIA MT 208, NO 715, SALA II, ANEXO A ARADO...",ALTA FLORESTA,MT,78.580-000,(66) 992477501,A2@SUPREMAAGRO.COM.BR,"PORTARIA NO 7867, DE 26 DE ABRIL DE 2022","AEROAGRICOLA, SERVICOS AEREOS PUBLICOS",26/04/2022 00:00:00,NaN,1001209
3,ABELHA TAXI AEREO E MANUTENCAO LTDA.,NAN,..,NAN,BRASIL,24.702.862/0001-24,"AV. GOV. PONCE A VIACUDA, S/NO - APTO. MARECHA...",VARZEA GRANDE,MT,78.110-900,(65) 3682-2389 - RAMAL 207 E 208 / 9972-9344,CTM@VOEABELHA.COM.BR,"PORTARIA NO 7994, DE 10 DE MAIO DE 2022","AEROMEDICO, TAXI-AEREO",10/05/2022 00:00:00,NaN,1001163
4,ABSA - AEROLINHAS BRASILEIRAS S.A.,LTG,M3,NAN,BRASIL,00.074.635/0001-33,"AEROPORTO INTERNACIONAL DE VIRACOPOS, RODOVIA ...",CAMPINAS,SP,13.052-970,(11) 5582-8055,GAR@TAM.COM.BR,"PORTARIA NO 9429, DE 05 DE OUTUBRO DE 2022","TRANSPORTE AEREO NAO REGULAR, TRANSPORTE AEREO...",10/10/2022 00:00:00,NaN,1000641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,WIP AVIACAO AGRICOLA LTDA,NAN,..,NAN,BRASIL,34.999.065/0001-04,"RUA PIO FERNANDES, N215, BAIRRO PARACATUZINHO",PARACATU,MG,38.603-352,(38) 99741-2321,WIP-AVIACAOAGRICOLA@UOL.COM.BR,PORTARIA NO 7830,AEROAGRICOLA,19/04/2022 00:00:00,NaN,1007463
818,"WORLD 2 FLY, S.L. - W2FLY",WFL,2W,LUISA MEDINA,ESPANHA,00.000.000/0000-00,"ALAMEDA SANTOS 2222, 2O ANDAR, CERQUEIRA CESAR",SAO PAULO,SP,01.418-200,000,LUISA@DICIERO.ADV.BR,OFICIO NO 191/2024/GEAM EMPRESAS/GEAM/SAS-ANAC...,ESTRANGEIRA NAO REGULAR,16/12/2024 00:00:00,15/11/2025 00:00:00,1007584
819,W R EBERLE,NAN,..,NAN,BRASIL,44.640.496/0001-33,"RUA 1, NO 50, QD 05 LT 10 A, CENTRO",MORRINHOS,GO,75.650-000,.,EBERLEAVIACAOAGRICOLA@HOTMAIL.COM,"PORTARIA NO 13.677, DE 23/01/2024",AEROAGRICOLA,29/01/2024 00:00:00,NaN,1007621
820,XINGU AERO AGRICOLA LTDA (ANTERIOR: RITTER AER...,NAN,..,NAN,BRASIL,07.629.863/0001-71,"AVENIDA PASTOR ALCIMAR BORGES, NO 137, QD. 77,...",ACREUNA,GO,75.960-000,(64) 999874441/ (64) 99618-4441,AERO.XINGU@GMAIL.COM,"PORTARIA NO 9.800, DE 21 DE NOVEMBRO DE 2022",AEROAGRICOLA,23/11/2022 00:00:00,NaN,1000194


In [74]:
df_vra = pd.read_csv('dados/raw/Dados Atividade/VRA_2024.csv', sep=';')
df_vra11 = pd.read_csv('dados/raw/Dados Voos Atualizados/VRA_2024_11.csv', sep=';')
df_vra12 = pd.read_csv('dados/raw/Dados Voos Atualizados/VRA_2024_12.csv', sep=';')

C:\Users\ivanm\AppData\Local\Temp\ipykernel_15856\4186212365.py:1: DtypeWarning: Columns (6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vra = pd.read_csv('dados/raw/Dados Atividade/VRA_2024.csv', sep=';')


In [ ]:
df_vra = padronizar_colunas(df_vra)
df_vra = remover_acentos_no_df(df_vra)
df_vra = padronizar_maiusculo(df_vra)
df_vra = padronizar_espacos_colunas(df_vra)
df_vra = padronizar_data02(df_vra,'REFERENCIA')
df_vra = df_vra[df_vra['SIGLA ICAO AEROPORTO DESTINO'].str.startswith('SB')]
df_vra = df_vra[df_vra['SIGLA ICAO AEROPORTO ORIGEM'].str.startswith('SB')]

In [ ]:
df_vra11 = padronizar_colunas(df_vra11  )
df_vra11 = remover_acentos_no_df(df_vra11)
df_vra11 = padronizar_maiusculo(df_vra11)
df_vra11 = padronizar_espacos_colunas(df_vra11)
df_vra11 = padronizar_data02(df_vra11,'REFERENCIA')
df_vra11 = df_vra11[df_vra11['SIGLA ICAO AEROPORTO DESTINO'].str.startswith('SB')]
df_vra11 = df_vra11[df_vra11['SIGLA ICAO AEROPORTO ORIGEM'].str.startswith('SB')]

In [ ]:
df_vra12 = padronizar_colunas(df_vra12  )
df_vra12 = remover_acentos_no_df(df_vra12)
df_vra12 = padronizar_maiusculo(df_vra12)
df_vra12 = padronizar_espacos_colunas(df_vra12)
df_vra12 = padronizar_data02(df_vra12,'REFERENCIA')
df_vra12 = df_vra12[df_vra12['SIGLA ICAO AEROPORTO DESTINO'].str.startswith('SB')]
df_vra12 = df_vra12[df_vra12['SIGLA ICAO AEROPORTO ORIGEM'].str.startswith('SB')]

In [78]:
#verificando se coluna é unica, ou seja, ID
df_vra['NUMERO VOO'].nunique() == len(df_vra)

False

In [79]:
df_main = pd.concat([df_vra, df_vra11, df_vra12], ignore_index=True)

In [80]:
print(len(df_main))

935895


In [81]:
chaves_unicas = [
    'SIGLA ICAO EMPRESA AEREA',
    'NUMERO VOO',
    'PARTIDA REAL' 
]

df_main = df_main.drop_duplicates(
    subset=chaves_unicas,
    keep='first'
)

In [82]:
print(len(df_main))

781558


In [83]:
df_aero

,CODIGO OACI,CIAD,NOME,MUNICIPIO ATENDIDO,UF,LATITUDE,LONGITUDE,ALTITUDE,OPERACAO,OACI,...,FREQ. SEMANAL,R1,R2,REFERENCIA(S),RESTRICAO(OES),SITUACAO,R1.1,REFERENCIA,SITUADO NA AMAZONIA LEGAL,OBSERVACOES
0,SBAA,PA0008,CONCEICAO DO ARAGUAIA,CONCEICAO DO ARAGUAIA,PA,8° 20' 55'' S,49° 18' 11'' W,199 M,VFR DIURNO/NOTURNO E IFR DIURNO,-,...,-,-,-,-,-,PROIBICAO DE OPERACOES DE POUSO DE AERONAVES D...,PA2019-1362,PA2019-1362,X,-
1,SBAE,SP0010,BAURU/AREALVA,BAURU,SP,22° 9' 28'' S,49° 4' 6'' W,598 M,VFR DIURNO/NOTURNO E IFR DIURNO/NOTURNO,-,...,-,-,-,-,-,-,-,-,NAN,-
2,SBAQ,SP0012,BARTOLOMEU DE GUSMAO,ARARAQUARA,SP,21° 48' 43'' S,48° 7' 59'' W,711 M,VFR DIURNO/NOTURNO E IFR DIURNO/NOTURNO,-,...,-,-,-,-,-,-,-,-,NAN,-
3,SBAR,SE0001,SANTA MARIA,ARACAJU,SE,10° 59' 7'' S,37° 4' 24'' W,7 M,VFR DIURNO/NOTURNO E IFR DIURNO/NOTURNO,RS,...,NAN,PA2022-7303 / PA2020-0406 / PA2021-4220,-,PA2022-7303 / PA2020-0406 / PA2021-4220,-,-,-,-,NAN,-
4,SBAT,MT0003,PILOTO OSVALDO MARQUES DIAS,ALTA FLORESTA,MT,9° 51' 59'' S,56° 6' 18'' W,289 M,VFR DIURNO/NOTURNO E IFR DIURNO/NOTURNO,-,...,-,-,-,-,-,-,-,-,X,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,SJO9,BA0253,SENHOR DO BONFIM,SENHOR DO BONFIM,BA,10° 31' 7'' S,40° 4' 55'' W,466 M,VFR DIURNO,-,...,-,-,-,-,-,-,-,-,NAN,-
500,SBPW,RJ0321,PORTO DO ACU,SAO JOAO DA BARRA,RJ,21° 48' 15'' S,41° 6' 32'' W,5 M,VFR DIURNO/NOTURNO E IFR DIURNO/NOTURNO,-,...,-,-,-,-,-,-,-,-,NAN,-
501,SSBG,RS0091,MUNICIPAL DE BENTO GONCALVES,BENTO GONCALVES,RS,29° 8' 51'' S,51° 32' 27'' W,671 M,VFR DIURNO,-,...,-,-,-,-,-,PROIBICAO DE OPERACOES DE POUSO DE AERONAVES T...,PA2024-14.530,PA2024-14.530,NAN,-
502,SND4,SC0020,ISMAEL NUNES,SAO JOAQUIM,SC,28° 14' 44'' S,49° 53' 51'' W,1426 M,VFR DIURNO,-,...,-,-,-,-,-,-,-,-,NAN,-


In [84]:
colunas_aero = ['CODIGO OACI', 'CIAD', 'NOME', 'MUNICIPIO ATENDIDO', 'UF', 'LATITUDE', 'LONGITUDE']
df_aero_filtrado = df_aero[colunas_aero].copy()    

In [85]:
# --- PASSO 1: JUNÇÃO PARA AEROPORTOS DE ORIGEM ---
# Primeiro, renomeia as colunas do df_aero_filtrado para forçar o sufixo ORIG
df_aero_orig = df_aero_filtrado.copy()
colunas_para_renomear = [col for col in df_aero_orig.columns if col != 'CODIGO OACI']
df_aero_orig = df_aero_orig.rename(columns={col: col + ' ORIG' for col in colunas_para_renomear})

df_main = pd.merge(
    df_main,
    df_aero_orig,
    how='left',
    left_on='SIGLA ICAO AEROPORTO ORIGEM',
    right_on='CODIGO OACI'
)

# --- PASSO 2: JUNÇÃO PARA AEROPORTOS DE DESTINO ---
# Renomeia as colunas do df_aero_filtrado para forçar o sufixo DEST
df_aero_dest = df_aero_filtrado.copy()
colunas_para_renomear = [col for col in df_aero_dest.columns if col != 'CODIGO OACI']
df_aero_dest = df_aero_dest.rename(columns={col: col + ' DEST' for col in colunas_para_renomear})

df_main = pd.merge(
    df_main,
    df_aero_dest,
    how='left',
    left_on='SIGLA ICAO AEROPORTO DESTINO',
    right_on='CODIGO OACI'
)

In [86]:
df_main

,SIGLA ICAO EMPRESA AEREA,EMPRESA AEREA,NUMERO VOO,CODIGO DI,CODIGO TIPO LINHA,MODELO EQUIPAMENTO,NUMERO DE ASSENTOS,SIGLA ICAO AEROPORTO ORIGEM,DESCRICAO AEROPORTO ORIGEM,PARTIDA PREVISTA,...,UF ORIG,LATITUDE ORIG,LONGITUDE ORIG,CODIGO OACI_y,CIAD DEST,NOME DEST,MUNICIPIO ATENDIDO DEST,UF DEST,LATITUDE DEST,LONGITUDE DEST
0,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5121,0,N,C208,9,SBSP,CONGONHAS - SAO PAULO - SP - BRASIL,01/01/2024 10:45,...,SP,23° 37' 34'' S,46° 39' 23'' W,SBJR,RJ0005,JACAREPAGUA - ROBERTO MARINHO,RIO DE JANEIRO,RJ,22° 59' 15'' S,43° 22' 12'' W
1,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5122,0,N,C208,9,SBJR,JACAREPAGUA - RIO DE JANEIRO - RJ - BRASIL,01/01/2024 12:45,...,RJ,22° 59' 15'' S,43° 22' 12'' W,SBSP,SP0001,SAO PAULO/CONGONHAS - DEPUTADO FREITAS NOBRE,SAO PAULO,SP,23° 37' 34'' S,46° 39' 23'' W
2,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5208,0,N,C208,9,SBSP,CONGONHAS - SAO PAULO - SP - BRASIL,01/01/2024 15:10,...,SP,23° 37' 34'' S,46° 39' 23'' W,SBJR,RJ0005,JACAREPAGUA - ROBERTO MARINHO,RIO DE JANEIRO,RJ,22° 59' 15'' S,43° 22' 12'' W
3,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5210,0,N,C208,9,SBJR,JACAREPAGUA - RIO DE JANEIRO - RJ - BRASIL,01/01/2024 17:05,...,RJ,22° 59' 15'' S,43° 22' 12'' W,SBSP,SP0001,SAO PAULO/CONGONHAS - DEPUTADO FREITAS NOBRE,SAO PAULO,SP,23° 37' 34'' S,46° 39' 23'' W
4,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5226,0,N,C208,9,SBSP,CONGONHAS - SAO PAULO - SP - BRASIL,01/01/2024 19:45,...,SP,23° 37' 34'' S,46° 39' 23'' W,SBJR,RJ0005,JACAREPAGUA - ROBERTO MARINHO,RIO DE JANEIRO,RJ,22° 59' 15'' S,43° 22' 12'' W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781553,UKL,UKRAINE AIR ALLIANCE AIRLINES,5005,6,I,AN12,0.0,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,16/12/2024 09:00,...,RJ,22° 48' 36'' S,43° 15' 2'' W,SBEG,AM0001,EDUARDO GOMES,MANAUS,AM,3° 2' 28'' S,60° 3' 2'' W
781554,UKL,UKRAINE AIR ALLIANCE AIRLINES,5016,9,X,AN12,0.0,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,NAN,...,AM,3° 2' 28'' S,60° 3' 2'' W,SBSJ,SP0008,PROFESSOR URBANO ERNESTO STUMPF,SAO JOSE DOS CAMPOS,SP,23° 13' 44'' S,45° 52' 16'' W
781555,UKL,UKRAINE AIR ALLIANCE AIRLINES,5016,2,I,AN12,0.0,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,25/12/2024 08:00,...,AM,3° 2' 28'' S,60° 3' 2'' W,SBSJ,SP0008,PROFESSOR URBANO ERNESTO STUMPF,SAO JOSE DOS CAMPOS,SP,23° 13' 44'' S,45° 52' 16'' W
781556,UKL,UKRAINE AIR ALLIANCE AIRLINES,5017,6,I,AN12,0.0,SBSJ,PROFESSOR URBANO ERNESTO STUMPF - SAO JOSE DOS...,26/12/2024 03:00,...,SP,23° 13' 44'' S,45° 52' 16'' W,SBEG,AM0001,EDUARDO GOMES,MANAUS,AM,3° 2' 28'' S,60° 3' 2'' W


In [87]:
def extrair_e_converter(coord_str):
    """Extrai os valores da string DMS e converte para graus decimais."""
    # Garante que o valor não seja nulo ou inválido
    if pd.isna(coord_str) or not isinstance(coord_str, str):
        return None
    
    # Usa uma expressão regular para extrair os números e a direção
    match = re.match(r"(\d+)°\s*(\d+)'\s*(\d+)''\s*([NSWE])", coord_str)
    if match:
        graus = int(match.group(1))
        minutos = int(match.group(2))
        segundos = int(match.group(3))
        direcao = match.group(4)
        return dms_para_dd(graus, minutos, segundos, direcao)
    return None



In [88]:
colunas_para_converter = [
    'LATITUDE ORIG', 'LONGITUDE ORIG', 
    'LATITUDE DEST', 'LONGITUDE DEST'
]

for col in colunas_para_converter:
    df_main[col] = df_main[col].apply(extrair_e_converter)


In [89]:
df_main['DISTANCIA KM'] = df_main.apply(
    lambda row: haversine(
        row['LATITUDE ORIG'], 
        row['LONGITUDE ORIG'], 
        row['LATITUDE DEST'], 
        row['LONGITUDE DEST']
    ), 
    axis=1
)


In [90]:
df_main

,SIGLA ICAO EMPRESA AEREA,EMPRESA AEREA,NUMERO VOO,CODIGO DI,CODIGO TIPO LINHA,MODELO EQUIPAMENTO,NUMERO DE ASSENTOS,SIGLA ICAO AEROPORTO ORIGEM,DESCRICAO AEROPORTO ORIGEM,PARTIDA PREVISTA,...,LATITUDE ORIG,LONGITUDE ORIG,CODIGO OACI_y,CIAD DEST,NOME DEST,MUNICIPIO ATENDIDO DEST,UF DEST,LATITUDE DEST,LONGITUDE DEST,DISTANCIA KM
0,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5121,0,N,C208,9,SBSP,CONGONHAS - SAO PAULO - SP - BRASIL,01/01/2024 10:45,...,-23.626111,-46.656389,SBJR,RJ0005,JACAREPAGUA - ROBERTO MARINHO,RIO DE JANEIRO,RJ,-22.987500,-43.370000,343.030915
1,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5122,0,N,C208,9,SBJR,JACAREPAGUA - RIO DE JANEIRO - RJ - BRASIL,01/01/2024 12:45,...,-22.987500,-43.370000,SBSP,SP0001,SAO PAULO/CONGONHAS - DEPUTADO FREITAS NOBRE,SAO PAULO,SP,-23.626111,-46.656389,343.030915
2,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5208,0,N,C208,9,SBSP,CONGONHAS - SAO PAULO - SP - BRASIL,01/01/2024 15:10,...,-23.626111,-46.656389,SBJR,RJ0005,JACAREPAGUA - ROBERTO MARINHO,RIO DE JANEIRO,RJ,-22.987500,-43.370000,343.030915
3,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5210,0,N,C208,9,SBJR,JACAREPAGUA - RIO DE JANEIRO - RJ - BRASIL,01/01/2024 17:05,...,-22.987500,-43.370000,SBSP,SP0001,SAO PAULO/CONGONHAS - DEPUTADO FREITAS NOBRE,SAO PAULO,SP,-23.626111,-46.656389,343.030915
4,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5226,0,N,C208,9,SBSP,CONGONHAS - SAO PAULO - SP - BRASIL,01/01/2024 19:45,...,-23.626111,-46.656389,SBJR,RJ0005,JACAREPAGUA - ROBERTO MARINHO,RIO DE JANEIRO,RJ,-22.987500,-43.370000,343.030915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781553,UKL,UKRAINE AIR ALLIANCE AIRLINES,5005,6,I,AN12,0.0,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,16/12/2024 09:00,...,-22.810000,-43.250556,SBEG,AM0001,EDUARDO GOMES,MANAUS,AM,-3.041111,-60.050556,2847.404725
781554,UKL,UKRAINE AIR ALLIANCE AIRLINES,5016,9,X,AN12,0.0,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,NAN,...,-3.041111,-60.050556,SBSJ,SP0008,PROFESSOR URBANO ERNESTO STUMPF,SAO JOSE DOS CAMPOS,SP,-23.228889,-45.871111,2714.305622
781555,UKL,UKRAINE AIR ALLIANCE AIRLINES,5016,2,I,AN12,0.0,SBEG,EDUARDO GOMES - MANAUS - AM - BRASIL,25/12/2024 08:00,...,-3.041111,-60.050556,SBSJ,SP0008,PROFESSOR URBANO ERNESTO STUMPF,SAO JOSE DOS CAMPOS,SP,-23.228889,-45.871111,2714.305622
781556,UKL,UKRAINE AIR ALLIANCE AIRLINES,5017,6,I,AN12,0.0,SBSJ,PROFESSOR URBANO ERNESTO STUMPF - SAO JOSE DOS...,26/12/2024 03:00,...,-23.228889,-45.871111,SBEG,AM0001,EDUARDO GOMES,MANAUS,AM,-3.041111,-60.050556,2714.305622


In [91]:
df_main.columns

Index(['SIGLA ICAO EMPRESA AEREA', 'EMPRESA AEREA', 'NUMERO VOO', 'CODIGO DI',
       'CODIGO TIPO LINHA', 'MODELO EQUIPAMENTO', 'NUMERO DE ASSENTOS',
       'SIGLA ICAO AEROPORTO ORIGEM', 'DESCRICAO AEROPORTO ORIGEM',
       'PARTIDA PREVISTA', 'PARTIDA REAL', 'SIGLA ICAO AEROPORTO DESTINO',
       'DESCRICAO AEROPORTO DESTINO', 'CHEGADA PREVISTA', 'CHEGADA REAL',
       'SITUACAO VOO', 'JUSTIFICATIVA', 'REFERENCIA', 'SITUACAO PARTIDA',
       'SITUACAO CHEGADA', 'CODIGO OACI_x', 'CIAD ORIG', 'NOME ORIG',
       'MUNICIPIO ATENDIDO ORIG', 'UF ORIG', 'LATITUDE ORIG', 'LONGITUDE ORIG',
       'CODIGO OACI_y', 'CIAD DEST', 'NOME DEST', 'MUNICIPIO ATENDIDO DEST',
       'UF DEST', 'LATITUDE DEST', 'LONGITUDE DEST', 'DISTANCIA KM'],
      dtype='object')

In [92]:
df_main.drop(columns=['DESCRICAO AEROPORTO ORIGEM', 'DESCRICAO AEROPORTO DESTINO'], inplace=True)

In [93]:
# Converte a coluna para o tipo datetime para permitir o particionamento
df_main['PARTIDA PREVISTA'] = pd.to_datetime(df_main['PARTIDA PREVISTA'], format='%d/%m/%Y %H:%M')

# Extrai o ano do DataFrame
ano = df_main['PARTIDA PREVISTA'].dt.year.iloc[0]

In [94]:
df_main

,SIGLA ICAO EMPRESA AEREA,EMPRESA AEREA,NUMERO VOO,CODIGO DI,CODIGO TIPO LINHA,MODELO EQUIPAMENTO,NUMERO DE ASSENTOS,SIGLA ICAO AEROPORTO ORIGEM,PARTIDA PREVISTA,PARTIDA REAL,...,LATITUDE ORIG,LONGITUDE ORIG,CODIGO OACI_y,CIAD DEST,NOME DEST,MUNICIPIO ATENDIDO DEST,UF DEST,LATITUDE DEST,LONGITUDE DEST,DISTANCIA KM
0,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5121,0,N,C208,9,SBSP,2024-01-01 10:45:00,01/01/2024 10:31,...,-23.626111,-46.656389,SBJR,RJ0005,JACAREPAGUA - ROBERTO MARINHO,RIO DE JANEIRO,RJ,-22.987500,-43.370000,343.030915
1,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5122,0,N,C208,9,SBJR,2024-01-01 12:45:00,01/01/2024 12:43,...,-22.987500,-43.370000,SBSP,SP0001,SAO PAULO/CONGONHAS - DEPUTADO FREITAS NOBRE,SAO PAULO,SP,-23.626111,-46.656389,343.030915
2,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5208,0,N,C208,9,SBSP,2024-01-01 15:10:00,01/01/2024 15:02,...,-23.626111,-46.656389,SBJR,RJ0005,JACAREPAGUA - ROBERTO MARINHO,RIO DE JANEIRO,RJ,-22.987500,-43.370000,343.030915
3,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5210,0,N,C208,9,SBJR,2024-01-01 17:05:00,01/01/2024 17:02,...,-22.987500,-43.370000,SBSP,SP0001,SAO PAULO/CONGONHAS - DEPUTADO FREITAS NOBRE,SAO PAULO,SP,-23.626111,-46.656389,343.030915
4,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5226,0,N,C208,9,SBSP,2024-01-01 19:45:00,01/01/2024 19:40,...,-23.626111,-46.656389,SBJR,RJ0005,JACAREPAGUA - ROBERTO MARINHO,RIO DE JANEIRO,RJ,-22.987500,-43.370000,343.030915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781553,UKL,UKRAINE AIR ALLIANCE AIRLINES,5005,6,I,AN12,0.0,SBGL,2024-12-16 09:00:00,16/12/2024 09:28,...,-22.810000,-43.250556,SBEG,AM0001,EDUARDO GOMES,MANAUS,AM,-3.041111,-60.050556,2847.404725
781554,UKL,UKRAINE AIR ALLIANCE AIRLINES,5016,9,X,AN12,0.0,SBEG,NaT,25/12/2024 08:15,...,-3.041111,-60.050556,SBSJ,SP0008,PROFESSOR URBANO ERNESTO STUMPF,SAO JOSE DOS CAMPOS,SP,-23.228889,-45.871111,2714.305622
781555,UKL,UKRAINE AIR ALLIANCE AIRLINES,5016,2,I,AN12,0.0,SBEG,2024-12-25 08:00:00,NAN,...,-3.041111,-60.050556,SBSJ,SP0008,PROFESSOR URBANO ERNESTO STUMPF,SAO JOSE DOS CAMPOS,SP,-23.228889,-45.871111,2714.305622
781556,UKL,UKRAINE AIR ALLIANCE AIRLINES,5017,6,I,AN12,0.0,SBSJ,2024-12-26 03:00:00,26/12/2024 10:44,...,-23.228889,-45.871111,SBEG,AM0001,EDUARDO GOMES,MANAUS,AM,-3.041111,-60.050556,2714.305622


In [95]:
novos_nomes = {
    'SIGLA ICAO AEROPORTO DESTINO' : 'SIGLA AEROPORTO DESTINO',
    'MUNICIPIO ATENDIDO ORIG' : 'MUNICIPIO ORIG',
    'MUNICIPIO ATENDIDO DEST' : 'MUNICIPIO DEST',
    'SIGLA ICAO AEROPORTO ORIGEM' : 'SIGLA AEROPORTO ORIGEM',
}

df_main.rename(columns=novos_nomes, inplace=True)

In [96]:
df_rec = pd.read_csv('dados/raw/Dados Atividade/dadosconsumidor2024.csv', sep=';', header=1)

In [97]:
df_rec

,Gestor,Região,UF,Cidade,Ano Abertura,Mês Abertura,Data Abertura,Data e Hora Resposta,Data e Hora Análise,Data e Hora Recusa,...,Problema,Forma Contrato,Procurou Empresa,Respondida,Situação,Avaliação Reclamação,Nota do Consumidor,Edição de Conteúdo,Interação do Gestor,Código Classificador ANAC
0,Secretaria Nacional do Consumidor,CO,DF,Brasília,2024,1,2024-01-01,2024-01-02 15:14:35,NaN,NaN,...,Oferta não cumprida / serviço não fornecido/ v...,Internet,S,S,Finalizada avaliada,Resolvida,4.0,N,N,S;I;W;D;P;0999
1,Secretaria Nacional do Consumidor,CO,DF,Brasília,2024,1,2024-01-01,2024-01-03 21:42:08,NaN,NaN,...,Má qualidade no atendimento presencial ou outr...,Internet,S,S,Finalizada não avaliada,Não Avaliada,NaN,N,N,N;M;SSA;D;P;4210
2,Secretaria Nacional do Consumidor,CO,DF,Brasília,2024,1,2024-01-01,2024-01-04 12:32:48,NaN,NaN,...,SAC - Demanda não resolvida / não respondida /...,Internet,S,S,Finalizada avaliada,Resolvida,5.0,N,N,S;I;G;D;P;1400
3,Secretaria Nacional do Consumidor,CO,DF,Brasília,2024,1,2024-01-01,2024-01-04 13:50:26,NaN,NaN,...,Dificuldade para contratar o serviço (problema...,Internet,S,S,Finalizada avaliada,Resolvida,5.0,N,N,S;P;W;D;P;8000/0710
4,Secretaria Nacional do Consumidor,CO,DF,Brasília,2024,1,2024-01-01,2024-01-05 11:20:26,NaN,NaN,...,"Extravio/ avaria de bagagem, carga, correspond...",Internet,S,S,Finalizada não avaliada,Não Avaliada,NaN,N,N,N;M;BSB;D;P;4120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90590,Fundação de Proteção e Defesa do Consumidor,SE,SP,Votuporanga,2024,9,2024-09-13,2024-09-13 17:41:10,NaN,NaN,...,Dificuldade / demora para cancelar o serviço,Internet,S,S,Finalizada avaliada,Resolvida,5.0,N,N,S;P;O;D;P;5110
90591,Fundação de Proteção e Defesa do Consumidor,SE,SP,Votuporanga,2024,9,2024-09-13,2024-09-13 18:40:48,NaN,NaN,...,Não entrega do contrato ou documentação relaci...,Internet,S,S,Finalizada avaliada,Resolvida,5.0,N,N,S;M;A;D;P;0505
90592,Fundação de Proteção e Defesa do Consumidor,SE,SP,Votuporanga,2024,11,2024-11-17,2024-11-19 14:43:50,NaN,NaN,...,Cancelamento de voo,Internet,S,S,Finalizada avaliada,Não Resolvida,1.0,N,N,S;P;CGH;D;P;2905
90593,Fundação de Proteção e Defesa do Consumidor,SE,SP,Votuporanga,2024,11,2024-11-19,2024-11-26 15:10:40,NaN,NaN,...,Cancelamento de voo,Internet,S,S,Finalizada não avaliada,Não Avaliada,NaN,N,N,S;P;O;D;P;2905


In [ ]:
df_rec = padronizar_colunas(df_rec)
df_rec = remover_acentos_no_df(df_rec)
df_rec = padronizar_maiusculo(df_rec)
df_rec = padronizar_espacos_colunas(df_rec)
df_rec = padronizar_data02(df_rec,'DATA ABERTURA')
df_rec = padronizar_data_hora(df_rec,'DATA E HORA RESPOSTA')
df_rec = padronizar_data02(df_rec,'DATA FINALIZACAO')
df_rec = padronizar_data02(df_rec,'PRAZO RESPOSTA')


In [99]:
df_rec.isnull().sum().sort_values(ascending=False)

PRAZO ANALISE GESTOR (EM DIAS)    89545
NOTA DO CONSUMIDOR                44084
DATA FINALIZACAO                  26998
DATA E HORA RESPOSTA                312
TEMPO RESPOSTA (EM DIAS)            312
CIDADE                                0
MES ABERTURA                          0
ANO ABERTURA                          0
DATA ABERTURA                         0
UF                                    0
GESTOR                                0
REGIAO                                0
PRAZO RESPOSTA                        0
DATA E HORA RECUSA                    0
DATA E HORA ANALISE                   0
NOME FANTASIA                         0
ASSUNTO                               0
GRUPO PROBLEMA                        0
PROBLEMA                              0
AREA                                  0
FORMA CONTRATO                        0
PROCUROU EMPRESA                      0
SITUACAO                              0
RESPONDIDA                            0
AVALIACAO RECLAMACAO                  0


In [100]:
print(len(df_rec))

90595


In [101]:
df_rec = df_rec.drop(columns=['PRAZO ANALISE GESTOR (EM DIAS)'])

## Exportando Arquivos tratados

In [102]:
for mes in range(1, 13):
    mes_str = f"{mes:02d}"  # Formato com 2 dígitos, preenchendo com zero à esquerda
   
    # Filtra o DataFrame para o mês atual
    df_mes = df_main[df_main['PARTIDA PREVISTA'].dt.month == mes].copy()
   
    if not df_mes.empty:
        # AQUI ESTÁ A CHAVE: converte a coluna de volta para string no formato desejado
        df_mes['PARTIDA PREVISTA'] = df_mes['PARTIDA PREVISTA'].dt.strftime('%d/%m/%Y %H:%M')
       
        # Salva o DataFrame filtrado em um arquivo CSV
        nome_arquivo = f'dados/trusted/voos/dados_voo_{int(ano)}{mes_str}.csv'  # Convertendo ano para int
        df_mes.to_csv(nome_arquivo, index=False)
       
        print(f"Arquivo '{nome_arquivo}' criado com sucesso.")
    else:
        print(f"Não há dados para o mês {mes_str}.")

Arquivo 'dados/trusted/voos/dados_voo_202401.csv' criado com sucesso.
Arquivo 'dados/trusted/voos/dados_voo_202402.csv' criado com sucesso.
Arquivo 'dados/trusted/voos/dados_voo_202403.csv' criado com sucesso.
Arquivo 'dados/trusted/voos/dados_voo_202404.csv' criado com sucesso.
Arquivo 'dados/trusted/voos/dados_voo_202405.csv' criado com sucesso.
Arquivo 'dados/trusted/voos/dados_voo_202406.csv' criado com sucesso.
Arquivo 'dados/trusted/voos/dados_voo_202407.csv' criado com sucesso.
Arquivo 'dados/trusted/voos/dados_voo_202408.csv' criado com sucesso.
Arquivo 'dados/trusted/voos/dados_voo_202409.csv' criado com sucesso.
Arquivo 'dados/trusted/voos/dados_voo_202410.csv' criado com sucesso.
Arquivo 'dados/trusted/voos/dados_voo_202411.csv' criado com sucesso.
Arquivo 'dados/trusted/voos/dados_voo_202412.csv' criado com sucesso.


In [103]:
# 1. Converte a coluna para o formato de data para podermos agrupar
df_rec['DATA ABERTURA'] = pd.to_datetime(df_rec['DATA ABERTURA'], format='%d/%m/%Y')

# 2. Agrupa o DataFrame por mês
grouped = df_rec.groupby(df_rec['DATA ABERTURA'].dt.to_period('M'))

# 3. Itera sobre cada grupo (cada mês)
for period, group_df in grouped:
    # Cria uma cópia para evitar o SettingWithCopyWarning
    df_mes = group_df.copy()

    # --- LINHA ADICIONADA AQUI ---
    # 4. Re-formata a coluna de data para o padrão DD/MM/AAAA antes de salvar
    df_mes['DATA ABERTURA'] = df_mes['DATA ABERTURA'].dt.strftime('%d/%m/%Y')
    
    # 5. Cria o nome do arquivo
    filename = f"dados/trusted/reclamacoes/reclamacoes{period.strftime('%Y%m')}.csv"

    # 6. Salva o grupo de dados no arquivo CSV
    df_mes.to_csv(filename, index=False)
    print(f"Arquivo '{filename}' criado com sucesso.")


Arquivo 'dados/trusted/reclamacoes/reclamacoes202401.csv' criado com sucesso.
Arquivo 'dados/trusted/reclamacoes/reclamacoes202402.csv' criado com sucesso.
Arquivo 'dados/trusted/reclamacoes/reclamacoes202403.csv' criado com sucesso.
Arquivo 'dados/trusted/reclamacoes/reclamacoes202404.csv' criado com sucesso.
Arquivo 'dados/trusted/reclamacoes/reclamacoes202405.csv' criado com sucesso.
Arquivo 'dados/trusted/reclamacoes/reclamacoes202406.csv' criado com sucesso.
Arquivo 'dados/trusted/reclamacoes/reclamacoes202407.csv' criado com sucesso.
Arquivo 'dados/trusted/reclamacoes/reclamacoes202408.csv' criado com sucesso.
Arquivo 'dados/trusted/reclamacoes/reclamacoes202409.csv' criado com sucesso.
Arquivo 'dados/trusted/reclamacoes/reclamacoes202410.csv' criado com sucesso.
Arquivo 'dados/trusted/reclamacoes/reclamacoes202411.csv' criado com sucesso.
Arquivo 'dados/trusted/reclamacoes/reclamacoes202412.csv' criado com sucesso.


In [104]:
df_empresas.to_csv('dados/trusted/empresas/empresas_tratadas.csv', index=False)